# Time Series Classification with aeon

Time Series Classification (TSC) involves training a model from a collection
 of time series (real valued, ordered, data) in order to predict a target variable.
 For example, we might want to build a model that can predict whether a patient
 is sick based on their ECG reading, or a persons type of movement based on the trace
  of the position of their hand. This notebook gives a quick guide to TSC to get you
  started using aeon time series classifiers. If you can use scikit-learn, it should
   be easy, because the basic usage is identical.

<img src="img/tsc.png" width="600" alt="time series classification">

## Classification Notebooks

This note book gives an overview of TSC. More specific notebooks on TSC are base on
the type of representation or transformation they use:

- [Convolution based](convolution_based.ipynb)
- [Deep learning](deep_learning.ipynb)
- [Dictionary based](dictionary_based.ipynb)
- [Distance based](distance_based.ipynb)
- [Feature based](feature_based.ipynb)
- [Interval based](interval_based.ipynb)
- [Shapelet based](shapelet_based.ipynb)
- [Hybrid](hybrid.ipynb)
- [Early classification](early_classification.ipynb)


## Data Storage and Problem Types

Time series can be univariate (each observation is a single value) or multivariate
(each observation is a vector). For example, an ECG reading from a single
sensor is a univariate series, but a motion trace of from a smart watch would be
multivariate, with at least three dimensions (x,y,z co-ordinates). The image above is
 a univariate problem: each series has its own label. The dimension of the time
 series instance is also often called the channel. We recommend storing time series
 in 3D numpy array of shape `(n_instances, n_channels, n_timepoints)` and
 where  possible our single problem loaders will return a
 3D numpy. Unequal length classification problems are stored in a list of 2D numpy
 arrays.

In [5]:
# # Plotting and data loading imports used in this notebook
# import warnings

# import matplotlib.pyplot as plt

# from aeon.datasets import load_arrow_head, load_basic_motions

# warnings.filterwarnings("ignore")

# arrow, arrow_labels = load_arrow_head(split="train")
# motions, motions_labels = load_basic_motions(split="train")
# print(f"ArrowHead series of type {type(arrow)} and shape {arrow.shape}")
# print(f"Motions type {type(motions)} of shape {motions_labels.shape}")

In [6]:
import numpy as np

trafficArr = np.loadtxt(open("traffic.csv", "rb"), delimiter=",")
labels = np.loadtxt(open("labels.csv", "rb"), delimiter=",", dtype=str)

# convert labels to numpy array
labels = np.array(labels)

# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(trafficArr, labels, test_size=0.2, random_state=42,stratify=labels)

from sklearn.metrics import classification_report

We tend to use 3D numpy even if the data is univariate, although all classifiers work
 with shape (instance, time point), currently some transformers do not work correctly
  with 2D arrays. If your series are unequal length, have missing values or are
  sampled at irregular time intervals, you should read the note book
  on [data preprocessing](../utils/preprocessing.ipynb).

The UCR/UEA [TSC dataset archive](https://timeseriesclassification.com/) contains a
large number of example TSC problems that have been used thousands of times in the
literature to assess TSC algorithms. These datasets have certain characteristics that
influence what data structure we use to store them in memory.

Most datasets in the archive contain time series all the same length. For example,
the [ArrowHead dataset](https://timeseriesclassification.com/description.php?Dataset=ArrowHead) we have just loaded consists of outlines of the images of
arrow heads. The classification of projectile points is an important topic in anthropology.

<img src="../img/arrow-heads.png" width="600" alt="arrow heads">

The shapes of the projectile points are converted into a sequence using the
angle-based method as described in this [blog post](https://izbicki.me/blog/converting-images-into-time-series-for-data-mining.html) about converting images into time series for data mining.

<img src="img/from-shapes-to-time-series.png" width="600" alt="from shapes to time series">

Each instance consists of a single time series (i.e. the problem is univariate) of
equal length and a class label based on shape distinctions such as the presence and
location of a notch in the arrow. The data set consists of 210 instances, by default split into 36 train and 175 test instances.


It is possible to use a standard `sklearn` classifier for univariate, equal length
classification problems, but it is unlikely to perform as well as bespoke time series
 classifiers, since `sklearn` classifiers ignore the sequence information in the variables.

To apply `sklearn` classifiers directly, the data needs to be reshaped into a 2D
numpy array. We also offer the ability to load univariate TSC problems directly in 2D
 arrays. Please note that currently,
 some Transformers treat a single multivariate time series in a numpy array as shape
 `(n_timepoints, n_channels)` rather than `(n_channels,n_timepoints)`
  do not work correctly  with 2D numpy classification problems, so we recommend using
   3D numpyof shape `(n_channels, 1, n_timepoints)` for univariate series.

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rand_forest = RandomForestClassifier(n_estimators=100)
rand_forest.fit(X_train, y_train)
y_pred = rand_forest.predict(X_test)

print("Accuracy:",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9557453416149069
               precision    recall  f1-score   support

   Google Doc       0.99      0.96      0.97       244
 Google Drive       0.95      0.96      0.95       327
 Google Music       0.91      0.90      0.91       118
Google Search       0.99      0.99      0.99       383
      Youtube       0.89      0.92      0.90       216

     accuracy                           0.96      1288
    macro avg       0.95      0.94      0.95      1288
 weighted avg       0.96      0.96      0.96      1288



## Time Series Classifiers in aeon

`aeon` contains the state of the art in time series classifiers in the package
`classification`. These are grouped based on the data representation used to find
discriminatory features. We provide a separate notebook for each of type:
[convolution based](convolution_based.ipynb), [deep learning](deep_learning.ipynb), [distance based](distance_based.ipynb), [dictionary based](dictionary_based.ipynb),
[feature_based](feature_based.ipynb), [hybrid](hybrid.ipynb), [interval based](interval_based.ipynb), and [shapelet based](shapelet_based.ipynb). We also
provide some
standard classifiers not available in scikit learn in the sklearn package.
We show the simplest use cases for classifiers and demonstrate how to build bespoke
pipelines for time series classification. An accurate and relatively
fast classifier is the [ROCKET](https://link.springer.com/article/10.1007/s10618-020-00701-z) classifier. ROCKET is a convolution based algorithm
described in detail in the [convolution based](convolution_based.ipynb) note book.

In [8]:
from aeon.classification.convolution_based import RocketClassifier

rocket = RocketClassifier(num_kernels=2000)
rocket.fit(X_train, y_train)
y_pred = rocket.predict(X_test)

print("Accuracy:",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

/Users/chenhajaj/miniconda3/envs/aeon-env/lib/python3.11/site-packages/aeon/transformations/collection/tsfresh.py:13: UserWarning: No module named 'tsfresh'. 'tsfresh' is a soft dependency and not included in the base aeon installation. Please run: `pip install tsfresh` to install the tsfresh package. To install all soft dependencies, run: `pip install aeon[all_extras]`
  _check_soft_dependencies("tsfresh", severity="warning")
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Accuracy: 0.9091614906832298
               precision    recall  f1-score   support

   Google Doc       0.99      0.97      0.98       244
 Google Drive       0.97      0.87      0.92       327
 Google Music       0.78      0.75      0.77       118
Google Search       0.98      0.96      0.97       383
      Youtube       0.72      0.89      0.80       216

     accuracy                           0.91      1288
    macro avg       0.89      0.89      0.89      1288
 weighted avg       0.92      0.91      0.91      1288



Another accurate classifier for time series classification is version 2 of the
[HIVE-COTE](https://link.springer.com/article/10.1007/s10994-021-06057-9) algorithm.
(HC2) is described in the [hybrid notebook](hybrid.ipynb) notebook. HC2 is relatively
slow
on small problems like these examples. However, it can be
configured with an approximate maximum run time as follows (it may take a bit longer
than 12 seconds to run this cell, very short times are approximate since there is a
minimum amount of work the classifier needs to do):

In [9]:
from aeon.classification.hybrid import HIVECOTEV2

hc2 = HIVECOTEV2(time_limit_in_minutes=0.2)
hc2.fit(X_train, y_train)
y_pred = hc2.predict(X_test)

print("Accuracy:",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9363354037267081
               precision    recall  f1-score   support

   Google Doc       0.99      0.97      0.98       244
 Google Drive       0.95      0.94      0.95       327
 Google Music       0.82      0.81      0.82       118
Google Search       0.99      0.97      0.98       383
      Youtube       0.84      0.89      0.86       216

     accuracy                           0.94      1288
    macro avg       0.92      0.92      0.92      1288
 weighted avg       0.94      0.94      0.94      1288



## Multivariate Classification
To use ``sklearn`` classifiers directly on multivariate data, one option is to flatten
the data so that the 3D array `(n_cases, n_channels, series_length)` becomes a 2D array
of shape `(n_cases, n_channels*series_length)`.

In [10]:
# motions_test, motions_test_labels = load_basic_motions(split="test")
# motions2d = motions.reshape(motions.shape[0], motions.shape[1] * motions.shape[2])
# motions2d_test = motions_test.reshape(
#     motions_test.shape[0], motions_test.shape[1] * motions_test.shape[2]
# )
# rand_forest.fit(motions2d, motions_labels)
# y_pred = rand_forest.predict(motions2d_test)
# accuracy_score(motions_test_labels, y_pred)

However, many ``aeon`` classifiers, including ROCKET and HC2, are configured to
work with multivariate input. This works exactly like univariate classification. For example:

In [11]:
# rocket.fit(motions, motions_labels)
# y_pred = rocket.predict(motions_test)
# accuracy_score(motions_test_labels, y_pred)

A list of classifiers capable of handling multivariate classification can be obtained
 with this code

In [12]:
from aeon.registry import all_estimators

all_estimators(
    filter_tags={"capability:multivariate": True},
    estimator_types="classifier",
    as_dataframe=True,
)

/Users/chenhajaj/miniconda3/envs/aeon-env/lib/python3.11/site-packages/aeon/networks/cnn.py:9: UserWarning: tensorflow and tensorflow-probability are required for deep learning and probabilistic functionality in `aeon`. To install these dependencies, run: `pip install aeon[dl]`
  _check_dl_dependencies(severity="warning")
/Users/chenhajaj/miniconda3/envs/aeon-env/lib/python3.11/site-packages/aeon/networks/encoder.py:9: UserWarning: tensorflow and tensorflow-probability are required for deep learning and probabilistic functionality in `aeon`. To install these dependencies, run: `pip install aeon[dl]`
  _check_dl_dependencies(severity="warning")
/Users/chenhajaj/miniconda3/envs/aeon-env/lib/python3.11/site-packages/aeon/networks/fcn.py:9: UserWarning: tensorflow and tensorflow-probability are required for deep learning and probabilistic functionality in `aeon`. To install these dependencies, run: `pip install aeon[dl]`
  _check_dl_dependencies(severity="warning")
/Users/chenhajaj/minicon

,name,estimator


An alternative for MTSC is to build a univariate classifier on each dimension, then
ensemble. Dimension ensembling can be easily done via ``ColumnEnsembleClassifier``
which fits classifiers independently to specified dimensions, then
combines predictions through a voting scheme. The interface is
similar to the ``ColumnTransformer`` from `sklearn`. The example below builds a DrCIF
classifier on the first channel and a RocketClassifier on the fourth and fifth
dimensions, ignoring the second, third and sixth.

In [13]:
# from aeon.classification.compose import ChannelEnsembleClassifier
# from aeon.classification.interval_based import DrCIFClassifier

# cls = ChannelEnsembleClassifier(
#     estimators=[
#         ("DrCIF0", DrCIFClassifier(n_estimators=5, n_intervals=2), [0]),
#         ("ROCKET3", RocketClassifier(num_kernels=1000), [3, 4]),
#     ]
# )

# cls.fit(motions, motions_labels)
# y_pred = cls.predict(motions_test)

# accuracy_score(motions_test_labels, y_pred)

## sklearn Compatibility

`aeon` classifiers are compatible with `sklearn` model selection and
composition tools using `aeon` data formats. For example, cross-validation can
be performed using the `sklearn` `cross_val_score` and `KFold` functionality:

In [14]:
from sklearn.model_selection import KFold, cross_val_score

cross_val_score(rocket, trafficArr, y=labels, cv=KFold(n_splits=4))

array([0.13913043, 0.3136646 , 0.87826087, 0.95152268])

Parameter tuning can be done using `sklearn` `GridSearchCV`. For example, we can tune
 the _k_ and distance measure for a K-NN classifier:

In [15]:
from sklearn.model_selection import GridSearchCV

from aeon.classification.distance_based import KNeighborsTimeSeriesClassifier

knn = KNeighborsTimeSeriesClassifier()
param_grid = {"n_neighbors": [1, 5], "distance": ["euclidean", "dtw"]}
parameter_tuning_method = GridSearchCV(knn, param_grid, cv=KFold(n_splits=4))

parameter_tuning_method.fit(X_train, y_train)
y_pred = parameter_tuning_method.predict(X_test)

accuracy_score(y_test, y_pred)

0.9433229813664596

Probability calibration is possible with the `sklearn` `CalibratedClassifierCV`:

In [16]:
from sklearn.calibration import CalibratedClassifierCV

from aeon.classification.interval_based import DrCIFClassifier

calibrated_drcif = CalibratedClassifierCV(
    estimator=DrCIFClassifier(n_estimators=10, n_intervals=5), cv=4
)

calibrated_drcif.fit(X_train, y_train)
y_pred = calibrated_drcif.predict(X_test)

accuracy_score(y_test, y_pred)

0.9510869565217391

### Background info and references for classifiers used here

#### KNeighborsTimeSeriesClassifier

One nearest neighbour (1-NN) classification with Dynamic Time Warping (DTW) is one of the oldest TSC approaches, and is commonly used as a performance benchmark.

#### RocketClassifier
The RocketClassifier is based on a pipeline combination of the ROCKET transformation (transformations.panel.rocket) and the sklearn RidgeClassifierCV classifier. The RocketClassifier is configurable to use variants MiniRocket and MultiRocket. ROCKET is based on generating random convolutional kernels. A large number are generated, then a linear classifier is built on the output.

[1] Dempster, Angus, François Petitjean, and Geoffrey I. Webb. "Rocket: exceptionally fast and accurate time series classification using random convolutional kernels." Data Mining and Knowledge Discovery (2020)
[arXiv version](https://arxiv.org/abs/1910.13051)
[DAMI 2020](https://link.springer.com/article/10.1007/s10618-020-00701-z)

#### DrCIF
The Diverse Representation Canonical Interval Forest Classifier (DrCIF) is an interval based classifier. The algorithm takes multiple randomised intervals from each series and extracts a range of features. These features are used to build a decision tree, which in turn are ensembled into a decision tree forest, in the style of a random forest.

Original CIF classifier:
[2] Matthew Middlehurst and James Large and Anthony Bagnall. "The Canonical Interval Forest (CIF) Classifier for Time Series Classification." IEEE International Conference on Big Data (2020)
[arXiv version](https://arxiv.org/abs/2008.09172)
[IEEE BigData (2020)](https://ieeexplore.ieee.org/abstract/document/9378424?casa_token=8g_IG5MLJZ4AAAAA:ItxW0bY4eCRwfdV9kLvf-8a8X73UFCYUGU9D19PwrHigjivLJVchxHwkM3Btn7vvlOJ_0HiLRa3LCA)

The DrCIF adjustment was proposed in [3].

#### HIVE-COTE 2.0 (HC2)
The HIerarchical VotE Collective of Transformation-based Ensembles is a meta ensemble that combines classifiers built on different representations. Version 2  combines DrCIF, TDE, an ensemble of RocketClassifiers called the Arsenal and the  ShapeletTransformClassifier. It is one of the most accurate classifiers on the UCR and UEA time series archives.

[3] Middlehurst, Matthew, James Large, Michael Flynn, Jason Lines, Aaron Bostrom, and Anthony Bagnall. "HIVE-COTE 2.0: a new meta ensemble for time series classification." Machine Learning (2021)
[ML 2021](https://link.springer.com/article/10.1007/s10994-021-06057-9)

#### Catch22

The CAnonical Time-series CHaracteristics (Catch22) are a set of 22 informative and low redundancy features extracted from time series data. The features were filtered from 4791 features in the `hctsa` toolkit.

[4] Lubba, Carl H., Sarab S. Sethi, Philip Knaute, Simon R. Schultz, Ben D. Fulcher, and Nick S. Jones. "catch22: Canonical time-series characteristics." Data Mining and Knowledge Discovery (2019)
[DAMI 2019](https://link.springer.com/article/10.1007/s10618-019-00647-x)